In [19]:
import nltk
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity      
import pandas as pd
import warnings
import streamlit as st 
warnings.filterwarnings('ignore')
# import spacy
lemmatizer = nltk.stem.WordNetLemmatizer()
# Download required NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/titokarika/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/titokarika/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/titokarika/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
data = pd.read_csv('Conversation 2.csv')
data.drop('Unnamed: 0', axis = 1, inplace = True)
data

,question,answer
0,"hi, how are you doing?",i'm fine. how about yourself?
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,no problem. so how have you been?,i've been great. what about you?
4,i've been great. what about you?,i've been good. i'm in school right now.
...,...,...
3720,that's a good question. maybe it's not old age.,are you right-handed?
3721,are you right-handed?,yes. all my life.
3722,yes. all my life.,you're wearing out your right hand. stop using...
3723,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.


In [21]:
# Define a function for text preprocessing (including lemmatization)
def preprocess_text(text):
    global tokens
    # Identifies all sentences in the data
    sentences = nltk.sent_tokenize(text)
    
    # Tokenize and lemmatize each word in each sentence
    preprocessed_sentences = []
    for sentence in sentences:
        tokens = [lemmatizer.lemmatize(word.lower()) for word in nltk.word_tokenize(sentence) if word.isalnum()]
        # Turns to basic root - each word in the tokenized word found in the tokenized sentence - if they are all alphanumeric 
        # The code above does the following:
        # Identifies every word in the sentence 
        # Turns it to a lower case 
        # Lemmatizes it if the word is alphanumeric

        preprocessed_sentence = ' '.join(tokens)
        preprocessed_sentences.append(preprocessed_sentence)
    
    return ' '.join(preprocessed_sentences)


data['tokenized question'] = data['question'].apply(preprocess_text)
data.head(20)

,question,answer,tokenized question
0,"hi, how are you doing?",i'm fine. how about yourself?,hi how are you doing
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.,i fine how about yourself
2,i'm pretty good. thanks for asking.,no problem. so how have you been?,i pretty good thanks for asking
3,no problem. so how have you been?,i've been great. what about you?,no problem so how have you been
4,i've been great. what about you?,i've been good. i'm in school right now.,i been great what about you
5,i've been good. i'm in school right now.,what school do you go to?,i been good i in school right now
6,what school do you go to?,i go to pcc.,what school do you go to
7,i go to pcc.,do you like it there?,i go to pcc
8,do you like it there?,it's okay. it's a really big campus.,do you like it there
9,it's okay. it's a really big campus.,good luck with school.,it okay it a really big campus


In [22]:
corpus = data['tokenized question'].to_list()
corpus

['hi how are you doing',
 'i fine how about yourself',
 'i pretty good thanks for asking',
 'no problem so how have you been',
 'i been great what about you',
 'i been good i in school right now',
 'what school do you go to',
 'i go to pcc',
 'do you like it there',
 'it okay it a really big campus',
 'good luck with school',
 'how it going',
 'i doing well how about you',
 'never better thanks',
 'so how have you been lately',
 'i actually been pretty good you',
 'i actually in school right now',
 'which school do you attend',
 'i attending pcc right now',
 'are you enjoying it there',
 'it not bad there are a lot of people there',
 'good luck with that',
 'how are you doing today',
 'i doing great what about you',
 'i absolutely lovely thank you',
 'everything been good with you',
 'i have been better how about yourself',
 'i started school recently',
 'where are you going to school',
 'i going to pcc',
 'how do you like it so far',
 'i like it so far my class are pretty good right n

In [23]:
tfidf_vector = TfidfVectorizer()
v_corpus = tfidf_vector.fit_transform(corpus)
print(v_corpus)

  (0, 509)	0.5065413071339148
  (0, 2031)	0.20246958926405542
  (0, 109)	0.3251863403401649
  (0, 852)	0.36497198982271084
  (0, 813)	0.6807872356994741
  (1, 2035)	0.605394945925918
  (1, 34)	0.3505537285608635
  (1, 648)	0.6202478098071607
  (1, 852)	0.3548262916820728
  (2, 122)	0.5527415789983398
  (2, 686)	0.31945428404430365
  (2, 1779)	0.5118018522872447
  (2, 747)	0.3500004798575619
  (2, 1347)	0.4560528865679781
  (3, 180)	0.4640078485793166
  (3, 791)	0.3196352277406537
  (3, 1615)	0.34970795213160755
  (3, 1352)	0.4966819887587219
  (3, 1162)	0.3646551029710526
  (3, 2031)	0.2061379998999602
  (3, 852)	0.3715846724193394
  (4, 1965)	0.34391223040871993
  (4, 759)	0.5651046183573343
  (4, 180)	0.5553723869987174
  (4, 34)	0.4393954404578242
  :	:
  (3721, 2031)	0.5285491886711198
  (3721, 109)	0.8489026770808895
  (3722, 1003)	0.6644852850034292
  (3722, 68)	0.4702691062201211
  (3722, 2026)	0.4141046481700366
  (3722, 1137)	0.4072144571510326
  (3723, 776)	0.3655399312419566

In [24]:
def bot_response(user_input):
    user_input_processed = preprocess_text(user_input)
    v_input = tfidf_vector.transform([user_input_processed])
    most_similar = cosine_similarity(v_input, v_corpus)
    most_similar_index = most_similar.argmax()
    
    return data['answer'].iloc[most_similar_index]

In [25]:
chatbot_greeting = [
    "Hello there, welcome to Conversations with Tito. pls ejoy your usage",
    "Hi user, This bot is created by Tito, Kindly enjoy your usage",
    "Hi hi, How you dey my nigga",
    "Alaye mi, Abeg enjoy your usage",
    "Hey Hey, pls enjoy your usage"    
]

user_greeting = ["hi", "hello there", "hey", "hi there"]
exit_word = ['bye', 'thanks bye', 'exit', 'goodbye']


print(f'\t\t\t\t\tWelcome To Conversations with Titos ChatBot\n\n')
while True:
    user_q = input('Pls Converse with Tito: ')
    if user_q in user_greeting:
        print(random.choice(chatbot_greeting))
    elif user_q in exit_word:
        print('Thank you for talking to me. Bye')
        break
    else:
        responses = bot_response(user_q)
        print(f'ChatBot:  {responses}')

					Welcome To Conversations with Titos ChatBot


Thank you for talking to me. Bye
